Data_Engineering_TIL_(20190409)


#### [학습목표]


- Elastic Search 기초개념 이해 및 실습


#### [학습기록]

#### Elastic Search 기초개념

- 최근 가장 많이 사용하고 있는 데이터 검색 엔진이다.


- 일라스틱 서치는 파이프라인 아키텍처에서 '분석 및 시각화' 쪽에 해당한다고 할 수 있다. 최근에 하둡과의 연동드라이버를 제공해서 하둡쪽의 데이터를 읽을 수 있게 제공도 해준다. 주용도는 원래 로그분석을 할때 가장 많이 쓴다. 데이터를 집어넣으면 바로 그래프를 뽑아주는 좋은 툴이다. 그리고 특정키에 대해 조회를 해도 바로바로 보여준다.


- 과거에는 ELK로 알려져 있었는데 최근에는 Elastic Stack으로 통용된다. 여러가지로 구성되는데 Elasticsearch, Kibana, Logstash, Beats가 포함된다.


- 처음에는 일라스틱 서치 다시말해 search 엔진으로 탄생했기 때문에 인덱싱하는 기능이 중요했다. 이렇게 하다보니 데이터를 찾는 것이 빨라지고 그러니까 '어 그러면 이런 결과를 대시보드에 보여주면 분석할때 더 좋겠다'라는 아이디어로 키바나가 탄생을 했다. 그래서 최근에는 키바나를 이용해서 빠르게 분석용할 수 있는 용도로 많이 쓰이고 있다.


- 일라스틱 서치가 일라스틱 스텍을 구성하는 메인 검색엔진이다. 로그스테시와 beats가 로그를 수집하기 위해서 사용하는 툴이다. 비츠가 조금 더 가벼운 편이고, 로그스테시가 메모리 2 ~ 3기가 정도 잡아먹는 헤비한 툴이다.


- 이 로그 수집툴들이 어떻게 작동하냐면, 서버들이(웹서비스를 하면 서버가 보통 몇개가 있다) 있으면 로그를 특정 폴더에다 떨굴것이다. 로그를 수집하기 위해서 로그스테시를 설치하고 그 고르가 떨궈지는 폴더를 소스로 설정하면, 로그스테시가 로그가 떨거지는 것을 사용자가 지정한 타겟으로 쏴주게 될 것이다. 타겟은 보통 일라스틱 서치고 이렇게 여러대의 서버에서 모아진 로그를 한군대에 모아서 분석하는 툴이 일라스틱 서치이다.

(아래 그림 참고)

<img src="7.png" width="1000" />

- 문제는 이 서버들마다 로그스테시를 깔았는데 서버들한테 영향을 주면 안된다. 왜냐하면 이 서버들의 주 목적은 서비스용이지 로그를 수집하는 용이 아니기 때문이다. 그래서 나온툴이 비츠다.


- 로그스테시는 또한 필터링 기능이 있다. 필터링이라는 것이 로그 포맷이라는 것이 표준화가 되어 있는데 그 표준화 항목들에 대해서 특정부분은 자를 수 있는 기능이 로그스테시에 탑재되어 있다. 그런데 이런 필터링을 추가하면 컴퓨팅을 하는 것이다. 이런 컴퓨팅이 서버에서 발생을하면 안그래도 메모리를 많이 차지하고 있는데 더욱더 로드가 찰것이다. 이런점을 주의해줘야 한다.


- 보통은 그래서 서버에 비츠를 깔고 로그스테시용 서버를 하나 둔 다음에 그 로그스테시용 서버에서 필터링을 하고 그 필터링 된 데이터를 일라스틱서치에 넣는 아래 그림처럼 아키텍쳐를 구성해서 운영을 하는 편이다.

<img src="8.png" width="1000" />

- Elastic Stack 주용도

1) Search 엔진

2) 빠른 데이터 분석

#### Elastic Search 실습

[실습프리뷰]

#### S3 버킷에서 로그스테시(특정한 서버를 가정한 EC2에서 설치해 구동)를 이용해서 로그데이터를 AWS 일라스틱 서치로 끌어와 간단한 분석해보기 

1) EC2 생성
   - EC2, AMI Linux 2, t2.medium, tag { Name: 자기이름 }, security group default

2) ssh 로 생성한 instance에 접속하기

3) java 및 logstash설치

4) logstash conf 파일 생성

(vi log1.conf 입력 및 실행, 그리고 아래의 내용 복붙)

In [ ]:
input {
s3 {
access_key_id => "xxxxxxxxxxxxxxxxx"
secret_access_key => "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
bucket => "xxxxxxx"
region => "ap-northeast-2"
prefix => "source/jsona/"

}
}
filter {
  json {
    source => "message"
  }
}
output {
  elasticsearch {
    hosts => "<자신의 ES 주소>:80"
    index => "logs-%{+YYYY.MM.dd}"
  }
 #  stdout { codec => rubydebug }
}

5) ES web UI 탐색 및 Dev Tools Console 설정

6) logstash 실행 

7) ES web UI -> Management -> Index Pattern에서 입수된 날짜 확인

8) Index Pattern 생성

(logs* 입력 후 넥스트 base_dt 선택후 클릭)

9) UI 탐색 & Visualization

[단계별 실습 상세내용]

step1) EC2 서비스 접속해서 launch instacne 클릭

<img src="1.png" width="1000" />

step2) 아래 그림처럼 맨위의 인스턴스 클릭

<img src="2.png" width="1000" />

step3) 대략 다음과 같은 사양으로 EC2를 생성해준다.

참고로 로그스테시가 메모리를 많이 먹기 때문에 사양이 높은것이 좋다.

<img src="3.png" width="1000" />

step 4) 배쉬쉘 또는 각종 터미널을 이용해서 생성한 EC2로 아래와 같이 접속해준다.

<img src="4.png" width="1000" />

step 5) 그 다음 우리는 자바를 설치해줘야 한다. 아래와 같은 명령어를 입력해준다.

$ sudo yum install -y java-1.8.0-openjdk-devel.x86_64

$ sudo /usr/sbin/alternatives --config java 

$ sudo /usr/sbin/alternatives --config javac


위의 명령어는 자바 버전이 두개이면 둘중에 어떤버전을 선택해서 쓸지 설정해주는 것인데 차피 하나의 버전에 자바만 선택할 것이기 때문에 이 명령어를 치고 나오는 후속문들은 전부 엔터를 쳐주면 된다.



$ sudo yum remove java-1.7.0-openjdk  -y

step 6) 그 다음 우리는 로그스테시를 설치해줘야 한다. 아래와 같은 명령어를 입력해준다.

$ wget https://artifacts.elastic.co/downloads/logstash/logstash-6.7.1.tar.gz

(위의 명령어는 로그 스테시를 홈페이지에서 다운로드 받는 명령어)

$ tar xvzf logstash-6.7.1.tar.gz

(tar 형식을 풀어주는 명령어)
 
$ ln –s ./logstash-6.7.1 logstash

step 7) vi ~/.bash_profile 명령어를 입력하고 아래와 같이 입력하여 쉘스크립트를 수정해준다.

<img src="4-2.png" width="500" />

step 8) 수정한 베쉬셀 재실행 명령어 입력

$ source ~/.bash_profile

step 9) vi log1.conf 입력 및 실행, 그리고 아래의 내용 복붙 및 저장

<img src="5.png" width="500" />

여기서 <자신의 ES 주소>는 아래 step 13) 의 빨간색 박스부분의 URL을 집어 넣어주면 된다.

그리고 vi log1.conf에서 설정할 수 있는 옵션들이 많다. 파싱에서 디테일한 부분이나 특정컬럼 날리는 문제 등을 설정해줄 수 있다.

참고로 일라스틱서치에서는 데이터가 너무 많이 쌓일 수 있는 job에서는 안하는것이 좋다. 데이터를 감당못하면 스케일업이나 스케일 아웃을 통해 확장을 시켜줘야 한다. 통상 1주일치, 1 ~ 2주치 데이터를 모니터링 하는것이 보통이다.

일라스틱 서치도 마스터노드와 데이터노드라는 클러스터로 구성되어 있다. 실습할때는 방금 만든 EC2 한대를 띄워서 하기 때문에 이 EC2가 마스터노드 데이터노드 둘다 역할을 수행할 것인데 원래는 여러대를 두고 클러스터링을 구성해서 하는 편이다.

일라스틱 서치를 설정하는 옵션을 보면 얘도 샤드의 개념이 있고 레플리카 개념이 있다. 샤드는 마스터샤드와 슬레이브 샤드로 구성되어 있다. 서치하는중에 클러스터 중 하나가 죽어버리면 이 서치하는 정보가 날아가기 때문에 레플리카라고 해서 여기를 통해 분산저장 및 처리할 수 있도록 되어있고, 빠른 서치속도를 위해서 샤드를 늘려줄 수 있는 구조로 되어있다.

대부분 인덱스라는 데이터베이스(일라스틱 서치를 위한 데이터베이스)를 만들게 되면 디폴트로 샤드를 5개, 레플리카 1개를 부여한다.

일라스틱 서치 디스크에 용량이 꽉차면 로그스테시에서 아무리 쏴도 안들어가기 때문에 잘 유지 시켜줘야 한다. 만약에 일라스틱 서치에 2주치 데이터를 유지하고 싶으면 날짜별로 하나씩 삭제할 수 있다. 그러면 그 만큼의 스페이스가 늘어나게 되고 일정기간의 데이터를 유지하는데 이 옵션을 잘 활용해야 한다.

step 10) AWS 콘솔에서 'Elasticsearch Service' 검색, 'create new domain' 클릭

step 11) 아래와 같이 설정 후 넥스트 클릭

참고로 최근에 일라스틱 서치 버전 6.5로 업데이트가 되었고 추가된 기능으로는 주기적으로 삭제하는 기능과 롤이라는 형태소 분석기가 들어갔다.

<img src="9.png" width="1000" />

step 12) 자기 아이디 입력하고, 인서트타입은 미디움, 인서트는 한개만 설정 후 넥스트 클릭

step 13) 퍼블릭 엑세스 옵션을 선택해준다.

그리고 이게 서치엔진이다 보니까 접근권한이 필요하다. 실습이다보니 어디서든 그냥 접근할 수 있도록 설정을 해주고 conform을 해주면 일라스틱 서치 서비스가 creation된다.

그리고 우리가 가져와서 실습으로 분석해볼 로그는 제이슨형식으로 쌓여진 데이터이다. 일단 파일들은 날짜별로 디렉토리가 구분되어 있고 시간별로 쌓여있는 것을 아래의 그림처럼 볼 수 있다.

jsona라고 되어있는 디렉토리가 원래 폴더명이고 그 밑에 서브폴더명이 파티셔닝한거라 보면 된다. 이런식으로 데이터가 쌓이고 있다고 가정한다.

이런식으로 데이터가 쌓이고 있으면 그 데이터를 일라스틱서치에서 데이터를 리얼타임으로 분석하기 위해 데이터를 일라스틱서치에 넣어주고 실제 대시보드에서 분석하는 실습을 진행할 것이다.

그리고 엑세스 폴리시에서는 다음과 같이 설정해준다.

<img src="9-2.png" width="1000" />

아까 생성한 EC2에서 이 밑의 그림의 S3버킷을 바라보면서 이 버킷에 있는 데이터를 읽어서 일라스틱서치에 쏠 것이다.

그러기 위해 로그스테시가 S3에 접근하려면 엑세스키와 시큐리티키가 필요하다. 그 관련 작업을 아까 log1.conf에 명시해준것이다.

<img src="10.png" width="1000" />

아까 만들어놓은 EC2의 로그스테시 자체에 신스디비라는 폴더가 있는데 그 폴더에 이 버킷에 있는 데이터중 어디까지 데이터를 가져왔는지 기록을 해놓는다. 그래서 로그스테시가 죽었다가 다시 깨어나도 그 기록을 보고 이어서 데이터를 가져올 수 있다. 내가 만약에 됐고 처음부터 다시 데이터를 가져오고 싶으면 그 신스디비라는 폴더를 삭제해주면된다.

step 13) 일라스틱 서치를 띄우면 다음과 같은 화면을 볼 수 있다.

Endpoint는 일라스틱 서치 접속 포인트이고, Kibana는 키바나를 구동할 수 있는 URL이다.

<img src="6.png" width="1000" />

step 14) 위의 그림에서 키바나 URL 클릭한다.

그러면 아래와 같은 인터페이스가 전시될 것이다.

참고로 일라스틱 서치에 데이터를 넣을때 데이터의 특정한 항목을 타임스템프 또는 데이트로 정의를 해야한다. 그거를 데이터를 일라스틱 서치로 넣어줄때 그렇게 정의를 해줘도 되고, 아니면 데이터를 그냥 다 넣고 일라스틱 서치에 쌓이는 위치에서 우리가 테이블을 크리에이션 하듯이 특정필드에 대해 정의를 해놓고, 그 필드로 데이터가 들어오면 어떤 데이터는 타임스템프야, 어떤 데이터는 데이트야 라고 미리 선언을 해줄 수 있다.

<img src="11.png" width="700" />

step 15) 화면 좌측 메뉴에서 Dev Tools 클릭

Dev Tools Console에서 아래 그림과 같이 수정하기

<img src="11-2.png" width="700" />

step 16) logstash -f log1.conf 명령어를 입력 및 실행하여 아래 그림과 같이 logstash를 구동한다.

<img src="11-3.png" width="700" />

step 17) 아래 그림과 같이 매니지먼트 클릭 -> 인덱스 패턴 클릭

<img src="12.png" width="700" />

인덱스 패턴은 우리가 생성한 인덱스를 볼 수 있는 메뉴이다.

데이터가 들어오면 확인할 수 있고, logs의 날짜들이 아래와 같이 전시가 될 것이다.

<img src="13.png" width="1000" />

step 18) 위의 그림에서 index pattern의 index-name에 logs* 를 입력하고 next step클릭하면 아래와 같은 그림이 전시될 것이다.

타임필드를 선택을하라는 옵션이 주어진다. 

맨꼭대기에 있는 옵션인 골뱅이 타임스템프는 이 logstash 이벤트 시간을 의미하는데 우리는 base_dt를 선택 할 것이다. 

그래서 base_dt를 선택하고 크리에이트 인덱스 패턴을 클릭해준다.

<img src="14.png" width="1000" />

step 19)

그러면 얘가 테이블 creation하듯이 쭉 보여줄 수 있게 된다.

이 create한 index pattern을 활용할 것이기 때문에 아래 그림과 같인 set as default(우측에 별모양 버튼)를 클릭해준다.

<img src="15a.png" width="1000" />

step 20) 이 키바나에서 기본적으로 데이터를 살펴보는 메뉴가 discovery이기 때문에 discovery로 접속해서 아래 그림과 같이 우측 상단에 시간조절옵션으로 자유롭게 시간대별로 데이터를 탐색해본다.

<img src="16a.png" width="1000" />

위와 같은 화면처럼 서비스 같은 것을 오픈하면 우리한테 발생하는 이벤트와 데이터 들어오는 것을 확인할 수 있다.

<img src="17.png" width="700" />

위의 그림과 같이 컬럼형태로도 볼 수 있고, 제이슨 형태로도 들어오는 것을 확인할 수 있다.

얘는 시간에 대해서 클릭을하면 알아서 그 시간대에 대한 디테일한 정보를 펼쳐서 보여준다. 그러면 시간대별로 추이를 확인하는데 용이할 것이다.

내가 특정한 서비스유저의 아이디에 대해서만 조회하고 싶다 또는 특정이벤트와 함께 같이 필터링해서 조회해보고 싶다. 이런것도 가능하다.

키바나에서 두가지 특정한 조건 검색하고 싶을때 아래와 같은 형식으로 입력하고 검색도 가능하다. 예를들어서

event_name:2b_screen_view AND adid:특정사용자아이디 

2b_screen_view는 event_name의 옵션중 하나이다

또한 키바나에서 디비처럼 쓸 수 있다 아래의 그림과 같이 드레그되어진 서치하는 명령어(get logs*/_search로 시작하는 코드들) 같은거를 썼을때 아래그림 우측화면과 같이 결과를 뽑아낼 수 있다.

<img src="17-2.png" width="700" />

실제 이정도 수준의 데이터를 키바나를 이용해서 위와 같이 전시해주는 것처럼 서버운영자들이 디비에서 그래픽형태로 전시한다고 구현을 시도한다면 상당히 오래 걸릴것이다.

데이터 수집단계에서 키네시스나 카프카나 이런것들은 일종에 큰 queue이다. s3에 저장할 수 있는 파이어호스라는 컨슈머를 달았었다. s3에 들어오는 데이터 중에서 특정한 데이터만 필터링한다고 하면 다시하나 빼서 firehose에 또는 로직을 넣기위해 람다 프로그램을 붙인다던지해서 그 특정데이터만 s3에 저장할 수 있다. 그래서 그 특정한 데이터만 일라스틱서치 같은 툴에서 모니터링 한다던지해서 특정업무에 실시간적으로 모니터링하고자 한다면 이런 방안도 있다.

키바나는 기타 여러가지 시각화 기능들이 제공한다.

히트맵에서 좌표는 시군구까지 위치데이터(위도경도 데이터) 시각화 기능도 제공한다.

<img src="18.png" width="700" />

<img src="19.png" width="700" />

<img src="20.png" width="700" />

<img src="21.png" width="700" />